# Visualization of all Goals and indicators
The idea of this section is to present a basic 
starting point for visualizing all SDGs in one dynamic tree graph
Two libraries are used: pandas and plotly.
Both come with extensive documentation and possibilities

pandas: https://pandas.pydata.org/docs/index.html

plotly treemaps: https://plotly.com/python/treemaps/

SDGs indicator list: https://unstats.un.org/sdgs/indicators/indicators-list/


This script should give you a very compact starting point for further exploration.
A minimal understanding of python is mandatory.

* how to handle missing data
* tree style visualizations
* example text processing


In [42]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go

In [43]:
!pwd

/Users/kienlef/Documents/Vorlesung/2022_SDG_Analytics/notebooks


# Simple Treemap example

In [44]:
import plotly.express as px
fig = px.treemap(
    names = ["Eve","Cain", "Seth", "Enos", "Noam", "Abel", "Awan", "Enoch", "Azura"],
    parents = ["", "Eve", "Eve", "Seth", "Seth", "Eve", "Eve", "Awan", "Eve"]
)
fig.update_traces(root_color="lightgrey")
fig.update_layout(margin = dict(t=50, l=25, r=25, b=25))
fig.show()

In [45]:
names = ["Eve","Cain", "Seth", "Enos", "Noam", "Abel", "Awan", "Enoch", "Azura"]
parents = ["", "Eve", "Eve", "Seth", "Seth", "Eve", "Eve", "Awan", "Eve"]
pd.DataFrame({'parents':parents,
              'names':names})

,parents,names
0,,Eve
1,Eve,Cain
2,Eve,Seth
3,Seth,Enos
4,Seth,Noam
5,Eve,Abel
6,Eve,Awan
7,Awan,Enoch
8,Eve,Azura


In [ ]:
# Visualize all SDG Goals and its indicators

In [46]:
my_topics=pd.read_excel('../data/Clean_tree_list_Global Indicator Framework after 2021 refinement_English.xlsx',sheet_name='indicators_flat')
my_topics['SDGs'][0]=''

my_topics

/opt/anaconda3/lib/python3.7/site-packages/openpyxl/worksheet/_reader.py:312: UserWarning:

Unknown extension is not supported and will be removed



,SDGs,Goals and Targets
0,,SDGs
1,SDGs,Goal 1. End poverty in all its forms everywhere
2,SDGs,"Goal 2. End hunger, achieve food security and ..."
3,SDGs,Goal 3. Ensure healthy lives and promote well-...
4,SDGs,Goal 4. Ensure inclusive and equitable quality...
...,...,...
509,"17.18 By 2020, enhance capacity-building suppo...",17.18.1 Statistical capacity indicator for Sus...
510,NaN,17.18.2 Number of countries that have national...
511,NaN,17.18.3 Number of countries with a national st...
512,"17.19 By 2030, build on existing initiatives t...",17.19.1 Dollar value of all resources made ava...


In [50]:
my_topics['Goals and Targets']=my_topics['Goals and Targets'].fillna(method='ffill')
my_topics['SDGs']=my_topics['SDGs'].fillna(method='ffill')
my_topics.shape

(514, 2)

In [51]:
my_topics=my_topics.drop_duplicates()
my_topics.shape

(435, 2)

# prepare hover text

In [55]:
## check for textwrap functionality
import textwrap
textwrap.wrap(my_topics['Goals and Targets'][18],20)

['1.1 By 2030,',
 'eradicate extreme',
 'poverty for all',
 'people everywhere,',
 'currently measured',
 'as people living on',
 'less than $1.25 a',
 'day']

In [56]:
def split_every_x_words(value,nr_chars):
    ''' the function is pushing the textwrap array in a concatenated string.
    Each element is separated by a line html line break element'''
    my_text=''
    #print(value)
    text_list=textwrap.wrap(value,nr_chars)
    for count,word in enumerate(text_list):     
        my_text+=(word+'<br>')      
    return my_text

In [58]:
my_labels=[split_every_x_words(each,20) for each in my_topics['Goals and Targets']]
my_parents=[split_every_x_words(each,20) for each in my_topics['SDGs']]

In [17]:
#example string with HTML elements to display within plotly plots
text_string='''<b>THis is a bold text<b> <br> Hyperlink <a href=”https://plot.ly/”> Chart Studio</a><br> with some new lines'''

# Treemap

In [60]:
## ensure theat the first parents element is empty 
my_parents[0]=''
my_labels[0]=my_parents[1]

fig = go.Figure(go.Treemap(
    labels=np.array(my_labels),
    parents=np.array(my_parents),
    hoverinfo= 'text',
    maxdepth=2,
    #marker=dict(
     #   colorscale='RdBu',
     #   cmid=0.5),
))

fig.update_layout(margin = dict(t=0, l=0, r=0, b=0),
                 uniformtext=dict(minsize=12, 
                                 # mode='show',
                                 ),
                  width=1200, height=800
                 )
fig.show()
fig.write_html("Overview_SDGs_treemap.html")

# SUNBURST

In [61]:
# use a special line split for the sunburst chart
my_labels=[split_every_x_words(each,35) for each in my_topics['Goals and Targets']]
my_parents=[split_every_x_words(each,35) for each in my_topics['SDGs']]


fig =go.Figure(go.Sunburst(
    labels=np.array(my_labels),
    parents=np.array(my_parents),
    insidetextorientation='radial',
    hoverinfo= 'text',
    maxdepth=6, #visualize the full depth

))


fig.update_layout(margin = dict(t=0, l=0, r=0, b=0),
                 uniformtext=dict(minsize=12, ),#mode='show'
                 width=1800, height=1200,   # very large graphic
                 hoverlabel = dict(
                    bgcolor = '#faf8de' )#whatever format you want
                 )

fig.show()
fig.write_html("Overview_SDGs_sunburst.html")
    